##### Introduction to KOMO and the Manipulation Framework

**What is KOMO ?**

KOMO, short for K-th Order Markov Optimization, is a powerful mathematical framework developed by Marc Toussaint. It's primarily used for solving complex optimization problems in robotics, particularly for tasks that involve sequential decision-making over time, such as motion planning and trajectory optimization.

In essence, KOMO allows us to define and solve optimization problems where the goal is to find the best possible sequence of actions (or states) that a robot should take to achieve a particular task. This involves optimizing an objective function while considering constraints over multiple time steps.

**Why use KOMO ?**

- Versatility: KOMO is highly versatile and can be applied to a wide range of problems in robotics, from simple motion planning to more complex, dynamic tasks.
- Flexibility: It allows the user to utilize objectives and constraints, making it adaptable to different scenarios and tasks.
- Efficiency: KOMO can efficiently solve large-scale problems that involve multiple variables and time steps.

**What is the Manipulation Framework?**

While KOMO is a powerful tool, it can be complex and challenging to use directly, especially for those new to optimization or robotics. To make KOMO more accessible and user-friendly, we have developed a manipulation framework. This framework acts as a high-level Python interface that simplifies the use of KOMO, making it easier to define and solve optimization problems without needing to delve into the lower-level details.

**Key Benefits of the Manipulation Framework**

1. Simplified Syntax: The framework provides a more intuitive and readable syntax for defining optimization problems, allowing you to focus on the problem itself rather than the underlying mechanics of KOMO.
2. Predefined Functions: It includes a set of predefined functions and templates for common tasks, reducing the amount of code you need to write.
3. Seamless Integration: The framework integrates seamlessly with RobotOperation (BotOP) framework, allowing you to easily incorporate KOMO into your robotics projects on a simulated or real robot.

**Example Workflow**

To give you a better idea of how this works, here's a simple workflow:

1. Define the Problem: Using the manipulation framework, you define the task the robot needs to accomplish, such as moving an object from one place to another.
2. Set Objectives and Constraints: Specify the objectives (e.g., minimize energy usage) and constraints (e.g., avoid obstacles) for the optimization problem.
3. Run the Optimization: The framework interfaces with KOMO to solve the optimization problem and generate an optimal sequence of actions.
4. Execute the Plan: The resulting plan can then be executed by the robot in a simulation or real-world environment.

**Conclusion**

By using the manipulation framework, you can harness the full power of KOMO without needing to become an expert in optimization techniques. This allows you to focus on designing and solving the actual robotics tasks, making your work both more efficient and more accessible. To illustrate this let us examine a few simple examples on how to use the manipulation framework, gradually increasing the complexity of the examples.

##### Pick and Place Manipulation

Before we can start doing any manipulation we need to:
1. Define a basic task environment
2. Define KOMO constraints via the manipulation framework. Since the manipulation framework.

**1. Task Environment - Pick and Place:**

We define a basic task environment with a single block and no obstacle for now by creating a config **C** as shown in the following cell.
The config file usually contains i) an agent or manipulator, which in our case is a single panda robot, ii) an object to manipulate i.e., a box and iii) definitions to conveniently access or define constraints with respect to the gripper, box, table and so on. 

In [11]:
import robotic as ry
import manipulation as manip
import numpy as np
import time
import random

In [12]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

C.addFrame("box", "table") \
    .setJoint(ry.JT.rigid) \
    .setShape(ry.ST.ssBox, [.15,.06,.06,.005]) \
    .setRelativePosition([-.0,.3-.055,.095]) \
    .setContact(1) \
    .setMass(.1)

C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box = "box";
table = "table";
boxSize = C.frame(box).getSize()

C.view()

0

**2. Top Grasp and Pick-And-Place:**

Using the manipulation framework we can efficently define a simple top grasp and pick-and-place scenario, where we i) perform a top grasp and ii) place the grasped box on the table. To examine the KOMO constraint for a specific action i.e., M.grasp_top_box you can right click on the function and examine its definition.

In [13]:
C.setJointState(qHome)
C.view_raise()

C.frame(box).setRelativePosition([-.0,.3-.055,.095])
C.frame(box).setRelativeQuaternion([1.,0,0,0])

for i in range(7):
        qStart = C.getJointState()

        graspDirection = 'yz'
        placeDirection = 'z'
        place_position = [(i%3)*.3-.3, .2]
        place_orientation = [-(i%2),((i+1)%2),0.]
        info = f'placement {i}: grasp {graspDirection} place {placeDirection} place_pos {place_position} place_ori {place_orientation}'
        print('===', info)

        M = manip.ManipulationModelling(C, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1, joint_limits=False)
        M.grasp_top_box(1., gripper, box, graspDirection)
        M.place_box(2., box, table, palm, placeDirection)
        M.target_relative_xy_position(2., box, table, place_position)
        M.target_x_orientation(2., box, place_orientation)
        M.solve()
        if not M.feasible:
                continue

        M2 = M.sub_motion(0)
        M2.retract([.0, .2], gripper)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.ret.feasible:
            continue

        M3 = M.sub_motion(1)
        M3.keep_distance([], table, box)
        M3.bias(.5, qHome, 1e0)
        M3.solve()
        if not M3.ret.feasible:
            continue
            
        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

del M

=== placement 0: grasp yz place z place_pos [-0.3, 0.2] place_ori [0, 1, 0.0]
  -- feasible:
     { time: 0.005603, evals: 18, done: 1, feasible: 1, sos: 0.0739073, f: 0, ineq: 0, eq: 6.83877e-05 }
  -- feasible:sub_motion_0--
     { time: 0.04104, evals: 28, done: 1, feasible: 1, sos: 1.02374, f: 0, ineq: 5.77999e-07, eq: 5.96935e-05 }
  -- feasible:sub_motion_1--
     { time: 0.014983, evals: 10, done: 1, feasible: 1, sos: 1.53593, f: 0, ineq: 0.000492064, eq: 0.00363825 }
=== placement 1: grasp yz place z place_pos [0.0, 0.2] place_ori [-1, 0, 0.0]
  -- feasible:
     { time: 0.064461, evals: 200, done: 1, feasible: 1, sos: 0.240815, f: 0, ineq: 0, eq: 0.131909 }
  -- feasible:sub_motion_0--
     { time: 0.081697, evals: 29, done: 1, feasible: 1, sos: 0.974625, f: 0, ineq: 2.62526e-06, eq: 0.000460238 }
  -- feasible:sub_motion_1--
     { time: 0.083786, evals: 34, done: 1, feasible: 1, sos: 5.72189, f: 0, ineq: 0.00170202, eq: 0.00355992 }
=== placement 2: grasp yz place z place_po

##### Pick and Place over an Obstacle Manipulation

To illustrate the ease of use for the manipulationframework, let us consider a small adjustment to the Pick and Place Manipulation, by adding an obstacle over which the agent has to navigate and place the box on the table. To do so we need to i) add an obstacle to the configuration and ii) add a distance constraint to ensure that the object being carried or the manipulator does not collide with the obstacle, which can be achieved by using the M.keep_distace function. Again to examine the KOMO constraints you can examine its definition by rightclicking on the function. 

**1. Task Environment - Pick and Place over Obstacle:**

In [14]:
import robotic as ry
import manipulation as manip
import numpy as np
import time
import random

In [15]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

C.addFrame("box", "table") \
    .setJoint(ry.JT.rigid) \
    .setShape(ry.ST.ssBox, [.15,.06,.06,.005]) \
    .setRelativePosition([-.0,.3-.055,.095]) \
    .setContact(1) \
    .setMass(.1)

# i) define the obstacle 
C.addFrame("obstacle", "table") \
    .setShape(ry.ST.ssBox, [.06,.15,.06,.005]) \
    .setColor([.1]) \
    .setRelativePosition([-.15,.3-.055,.095]) \
    .setContact(1)

C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box = "box";
table = "table";
boxSize = C.frame(box).getSize()

C.view()

0

**2. Top Grasp and Pick-And-Place over Obstacle:**


In [16]:
C.setJointState(qHome)
C.view_raise()

C.frame(box).setRelativePosition([-.0,.3-.055,.095])
C.frame(box).setRelativeQuaternion([1.,0,0,0])

for i in range(7):
        qStart = C.getJointState()

        graspDirection = 'yz' #random.choice(['xz', 'yz'])
        placeDirection = 'z'
        place_position = [(i%3)*.3-.3, .2]
        place_orientation = [-(i%2),((i+1)%2),0.]
        info = f'placement {i}: grasp {graspDirection} place {placeDirection} place_pos {place_position} place_ori {place_orientation}'
        print('===', info)

        M = manip.ManipulationModelling(C, helpers=[gripper])
        M.setup_pick_and_place_waypoints(gripper, box, homing_scale=1e-1, joint_limits=False)
        M.grasp_top_box(1., gripper, box, graspDirection)
        M.place_box(2., box, table, palm, placeDirection)
        M.target_relative_xy_position(2., box, table, place_position)
        M.target_x_orientation(2., box, place_orientation)
        M.solve()
        if not M.feasible:
                continue

        M2 = M.sub_motion(0)
        M2.retract([.0, .2], gripper)
        M2.approach([.8, 1.], gripper)
        M2.solve()
        if not M2.ret.feasible:
            continue

        M3 = M.sub_motion(1)
        # ii) distance constrain between the table and box
        # ii) and a distance constrain so that box and obstacle dont collide 
        M3.keep_distance([], table, box)
        M3.keep_distance([], box, 'obstacle')
        M3.bias(.5, qHome, 1e0)
        M3.solve()
        if not M3.ret.feasible:
            continue
            
        M2.play(C)
        C.attach(gripper, box)
        M3.play(C)
        C.attach(table, box)

del M

=== placement 0: grasp yz place z place_pos [-0.3, 0.2] place_ori [0, 1, 0.0]
  -- feasible:
     { time: 0.005802, evals: 18, done: 1, feasible: 1, sos: 0.0739073, f: 0, ineq: 0, eq: 6.83843e-05 }
  -- feasible:sub_motion_0--
     { time: 0.041592, evals: 29, done: 1, feasible: 1, sos: 1.02374, f: 0, ineq: 5.78554e-07, eq: 5.97467e-05 }
  -- feasible:sub_motion_1--
     { time: 0.033777, evals: 18, done: 1, feasible: 1, sos: 1.54354, f: 0, ineq: 0.000492129, eq: 0.00363911 }
=== placement 1: grasp yz place z place_pos [0.0, 0.2] place_ori [-1, 0, 0.0]
  -- feasible:
     { time: 0.062057, evals: 200, done: 1, feasible: 1, sos: 0.240737, f: 0, ineq: 0, eq: 0.131645 }
  -- feasible:sub_motion_0--
     { time: 0.029631, evals: 14, done: 1, feasible: 1, sos: 0.974527, f: 0, ineq: 2.625e-06, eq: 0.000460174 }
  -- feasible:sub_motion_1--
     { time: 0.07435, evals: 39, done: 1, feasible: 1, sos: 5.73344, f: 0, ineq: 0.00165581, eq: 0.0038466 }
=== placement 2: grasp yz place z place_pos [

##### Random Pushes Manipulation

Another primitive manipulation task we can define is a random push manipulation, where the manipulator will randomly push an obstacle i.e., a box around on the table. Again we need to define the config file usually contains i) an agent or manipulator i.e., a single panda robot, ii) an object to manipulate or push in this case i.e., a box and optionally iii) definitions to conveniently access or define constraints with respect to the gripper, box, table and so on. 

**1. Task Environment - Random Pushes:**

In [17]:
import robotic as ry
import manipulation as manip
import numpy as np

In [19]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

C.addFrame("box", "table") \
    .setJoint(ry.JT.rigid) \
    .setShape(ry.ST.ssBox, [.15,.06,.06,.005]) \
    .setRelativePosition([-.0,.3-.055,.095]) \
    .setContact(1) \
    .setMass(.1)

C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper";
palm = "l_palm";
box = "box";
obj = box;
table = "table";
boxSize = C.frame(box).getSize()

C.view()

0

**2. Random Pushes:**

In [20]:
C.frame("l_panda_finger_joint1").setJointState(np.array([.0]))

#obj = box
C.frame(obj).setRelativePosition([-.0,.3-.055,.095])
C.frame(obj).setRelativeQuaternion([1.,0,0,0])

for i in range(30):
     qStart = C.getJointState()

     info = f'push {i}'
     print('===', info)

     M = manip.ManipulationModelling(C, info, ['l_gripper'])
     M.setup_pick_and_place_waypoints(gripper, obj, 1e-1, accumulated_collisions=False)
     print(type(obj))
     print(type(gripper))
     print(type(table))
     print(obj)
     print(gripper)
     print(table)
     M.straight_push([1.,2.], obj, gripper, table)
     #random target position
     M.komo.addObjective([2.], ry.FS.position, [obj], ry.OT.eq, 1e1*np.array([[1,0,0],[0,1,0]]), .4*np.random.rand(3) - .2+np.array([.0,.3,.0]))
     M.solve()
     if not M.ret.feasible:
          continue

     M1 = M.sub_motion(0, accumulated_collisions=False)
     M1.retractPush([.0, .15], gripper, .03)
     M1.approachPush([.85, 1.], gripper, .03)
     M1.keep_distance([.15,.85], obj, "l_finger1", .02)
     M1.keep_distance([.15,.85], obj, "l_finger2", .02)
     M1.keep_distance([.15,.85], obj, 'l_palm', .02)
     M1.keep_distance([], table, "l_finger1", .0)
     M1.keep_distance([], table, "l_finger2", .0)
     M1.solve()
     if not M1.ret.feasible:
          continue

     M2 = M.sub_motion(1, accumulated_collisions=False)
     M2.komo.addObjective([], ry.FS.positionRel, [gripper, '_push_start'], ry.OT.eq, 1e1*np.array([[1,0,0],[0,0,1]]))
     M2.solve()
     if not M2.ret.feasible:
          continue

     M1.play(C, 1.)
     C.attach(gripper, obj)
     M2.play(C, 1.)
     C.attach(table, obj)

del M

=== push 0
<class 'str'>
<class 'str'>
<class 'str'>
box
l_gripper
table
  -- feasible:push 0
     { time: 0.01202, evals: 59, done: 1, feasible: 1, sos: 0.29513, f: 0, ineq: 0, eq: 3.53594e-06 }
  -- feasible:sub_motion_0--push 0
     { time: 0.063333, evals: 52, done: 1, feasible: 1, sos: 1.86278, f: 0, ineq: 0, eq: 0.000183832 }
  -- feasible:sub_motion_1--push 0
     { time: 0.008381, evals: 11, done: 1, feasible: 1, sos: 0.0625385, f: 0, ineq: 0, eq: 0.000179277 }
=== push 1
<class 'str'>
<class 'str'>
<class 'str'>
box
l_gripper
table
  -- feasible:push 1
     { time: 0.013065, evals: 72, done: 1, feasible: 1, sos: 0.33418, f: 0, ineq: 1.08091e-07, eq: 9.46389e-07 }
  -- feasible:sub_motion_0--push 1
     { time: 0.049719, evals: 39, done: 1, feasible: 1, sos: 0.934721, f: 0, ineq: 2.60593e-07, eq: 0.000109226 }
  -- feasible:sub_motion_1--push 1
     { time: 0.010909, evals: 14, done: 1, feasible: 1, sos: 0.0983222, f: 0, ineq: 0, eq: 0.000155386 }
=== push 2
<class 'str'>
<clas

##### Bridge Building Manipulation

A more advanced manipulation task we can define using the manipulation framework is a bridge building manipulation, where we have three blocks and need to place two blocks next to each other with a single last block ontop of the other two, forming some sort of bridge or gate. Again we need to define the config file usually contains i) an agent or manipulator i.e., a single panda robot, ii) three blocks to manipulate or in this case to pick and place iii) definitions to conveniently access or define constraints with respect to the gripper, box, table and so on. 

**1. Task Environment - Bridge Building:**

In [21]:
import robotic as ry
import manipulation as manip
import numpy as np
import random

In [22]:
def create_n_boxes(C, N, position='fixed'):

    color_map = [[1,0,.5], # pink
                [0.5,1,0], # lime green
                [.5,1,1], # light turqois
                [0.5,0,1], # violet
                [1,0,0], # red
                [0,1,0], # green
                [0,0,1], # blue
                [1,.5,0], # orange
                [1,1,.5], # lime yellow
                [1,.5,1], # light pink                            
                ]

    for i in range(N):
        box_name = 'box{}'.format(i + 1)

        if position == 'fixed':
            position_val1 = 0.13 * (i - 2.5)
            C.addFrame(box_name) \
                .setPosition([position_val1, 0.15, 0.705]) \
                .setShape(ry.ST.ssBox, size=[0.04, 0.04, 0.12, 0.001]) \
                .setColor(color_map[i % len(color_map)]) \
                .setContact(True) \
                .setMass(1e-2)
    
    return C

In [23]:
C = ry.Config()
C.addFile(ry.raiPath("scenarios/pandaSingle.g"))
n_boxes = 3
keep_dist_between = ["box1", "box2", "box3"]
C = create_n_boxes(C=C, N=n_boxes, position="fixed")

# remove so collision with wrist camera is ignored
C.delFrame("panda_collCameraWrist")

# for convenience, a few definitions:
qHome = C.getJointState()
gripper = "l_gripper"
palm = "l_palm"
box1 = "box1"
box2 = "box2"
box3 = "box3"
table = "table"
boxSize = C.frame(box1).getSize()

C.view()


0

In [24]:
C.setJointState(qHome)
C.view_raise()
boxes_pos = [
    C.frame(box1).getPosition(),
    C.frame(box2).getPosition(),
    C.frame(box3).getPosition(),
]
boxes = [box1, box2, box3]
last_box_memory = []
n_success = 0
while n_success <= 2:
    box = np.random.choice(boxes)
    qStart = C.getJointState()
    graspDirection = random.choice(["x", "y"])
    placeDirection = "z"
    info = f"placement {n_success}: grasp {graspDirection} place {placeDirection}"
    print("===", info)

    M = manip.ManipulationModelling(C, info, helpers=[gripper])
    M.setup_pick_and_place_waypoints(
        gripper, box, homing_scale=1.5e-1, accumulated_collisions=True
    )
    M.grasp_box(1.0, gripper, box, palm, graspDirection)
    print("place direction:", placeDirection)

    # if the second block has been placed sucessfully
    if n_success > 1:

        print(f"3rd Block: ({box})")

        # # Calculate the direction vector by subtracting the position of the most recent object
        # from the position of the second most recent object. This gives the direction from
        # the last object to the second last object.
        dir_vec = (
            C.getFrame(last_box_memory[-2]).getPosition()
            - C.getFrame(last_box_memory[-1]).getPosition()
        )
        # Store the positions of the second last and last objects in `pos2` and `pos1`, respectively,
        # for clarity and reuse in later calculations.
        pos2 = C.getFrame(last_box_memory[-2]).getPosition()
        pos1 = C.getFrame(last_box_memory[-1]).getPosition()
        # Calculate the midpoint between the two positions (average of `pos1` and `pos2`), then add a small
        # vertical offset to slightly raise the position along the z-axis.
        rel_pos = (pos1 + pos2) / 2 + np.array([0, 0, 0.080])
        # Normalize the direction vector to have magnitude of 1.
        # `dir_vec` now only represents the direction and not the distance between
        # the two objects. Important for correctly aligning the orientation of the `box`
        # in the subsequent objective, without affecting the scale.
        dir_vec /= np.linalg.norm(dir_vec)

        M.komo.addObjective([2.0], ry.FS.position, [box], ry.OT.eq, [1e1], rel_pos)
        # Add an objective to align the z-axis (or another relevant axis) of `box` with the
        # of `dir_vec`. Since `dir_vec` is normalized, it ensures correct orientation
        # alignment without any unintended scaling effects.
        M.komo.addObjective([2.0], ry.FS.vectorZ, [box], ry.OT.eq, [1e1], dir_vec)

        print("Grasp Direction:", graspDirection)

        if graspDirection == "x":
            M.komo.addObjective(
                [2.0], ry.FS.scalarProductXZ, [box, table], ry.OT.eq, [1e1], [0.0]
            )
        elif graspDirection == "y":
            M.komo.addObjective(
                [2.0], ry.FS.scalarProductYZ, [box, table], ry.OT.eq, [1e1], [0.0]
            )
        M.keep_distance([], last_box_memory[-1], gripper)
        M.keep_distance([], last_box_memory[-2], gripper)

    # if the first block has been sucessfuly placed    
    if n_success == 1:
        print(f"2nd Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)

        M.komo.addObjective(
            [2.0],
            ry.FS.negDistance,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [-0.04],
        )
        M.komo.addObjective(
            [2.0],
            ry.FS.scalarProductXY,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1e1],
            [0],
        )
        M.komo.addObjective(
            [2.0],
            ry.FS.positionRel,
            [box, last_box_memory[-1]],
            ry.OT.eq,
            [1.0],
            [0.0, 0.0, 0.1],
        )
        # keep distance between gripper and previously placed box
        M.keep_distance([], last_box_memory[-1], gripper)

    M.keep_distance([], palm, table)
   
    # if the first block has not been placed yet i.e, no sucesses
    if n_success == 0:
        print(f"1st Block: ({box})")
        M.place_box(2.0, box, table, palm, placeDirection)
        # place inital block as chosen by np.random.choice at a random location
        block_pos = [np.random.uniform(0.0, 0.1), np.random.uniform(0.05, 0.25)]
        M.target_relative_xy_position(2.0, box, table, block_pos)

    ways = M.solve(verbose=1)

    if not M.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M :("
        )
        continue
    print("Submotion 0")

    M2 = M.sub_motion(0)

    M2.keep_distance([0.3, 0.7], palm, box)

    M2.retract([0.0, 0.2], gripper, dist=0.13)
    M2.approach([0.8, 1.0], gripper)
    M2.solve(verbose=1)
    if not M2.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M2 :("
        )
        continue

    M3 = M.sub_motion(1)

    M3.keep_distance([], table, box)
    
    if n_success > 1:
        M3.retract([0.0, 0.2], gripper, dist=0.13)
        M3.approach([0.8, 1.0], gripper)


    M3.solve(verbose=1)
    if not M3.ret.feasible:
        print(
            f"Failed placing block number {n_success+1} with grasp direction {graspDirection} at M3 :("
        )
        continue

    M2.play(C)
    C.attach(gripper, box)
    M3.play(C)
    C.attach(table, box)
    # update memory
    last_box_memory.append(box)
    # remove box that was previously used
    boxes.remove(box)
    # update positions
    boxes_pos = [
        C.frame(box1).getPosition(),
        C.frame(box2).getPosition(),
        C.frame(box3).getPosition(),
    ]
    n_success += 1

del M

=== placement 0: grasp y place z
place direction: z
1st Block: (box3)
  -- feasible:placement 0: grasp y place z
     { time: 0.050583, evals: 113, done: 1, feasible: 1, sos: 0.0893642, f: 0, ineq: 0.00470963, eq: 0.284504 }
Submotion 0
  -- feasible:sub_motion_0--placement 0: grasp y place z
     { time: 0.067078, evals: 44, done: 1, feasible: 1, sos: 3.08677, f: 0, ineq: 1.4792e-06, eq: 0.320344 }
  -- feasible:sub_motion_1--placement 0: grasp y place z
     { time: 0.014066, evals: 7, done: 1, feasible: 1, sos: 0.0113441, f: 0, ineq: 0.00487019, eq: 0.323102 }
=== placement 1: grasp y place z
place direction: z
2nd Block: (box2)
  -- feasible:placement 1: grasp y place z
     { time: 0.041714, evals: 100, done: 1, feasible: 1, sos: 0.071328, f: 0, ineq: 0.00787271, eq: 0.497486 }
Submotion 0
  -- feasible:sub_motion_0--placement 1: grasp y place z
     { time: 0.059329, evals: 29, done: 1, feasible: 1, sos: 2.96162, f: 0, ineq: 1.00961e-06, eq: 0.2941 }
  -- feasible:sub_motion_1--p